In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F

# Load data
df = pd.read_csv("p1_emg.csv")  # Replace with actual filename


In [2]:
# Label mapping (excluding 'rest')
mapg = {
    "Thumb Extension":0,"index Extension":1,"Middle Extension":2,"Ring Extension":3,
    "Pinky Extension":4,"Thumbs Up":5,"Right Angle":6,"Peace":7,"OK":8,"Horn":9,"Hang Loose":10,
    "Power Grip":11,"Hand Open":12,"Wrist Extension":13,"Wrist Flexion":14,
    "Ulnar deviation":15,"Radial Deviation":16
}

df = df[df['label'] != 'rest']
df['label'] = df['label'].map(mapg)

# Normalize EMG features
features = [col for col in df.columns if 'emg' in col]
# Min-Max Scaling to [0, 1]
min_vals = df[features].min()
max_vals = df[features].max()
df[features] = (df[features] - min_vals) / (max_vals - min_vals + 1e-8)


# Parameters
window_size = 100  # adjust based on your data
features = [col for col in df.columns if 'emg' in col]

# Create clean, non-overlapping windows with consistent labels
X = []
y = []

for i in range(0, len(df) - window_size + 1, window_size):
    window = df.iloc[i:i+window_size]
    label_set = window['label'].unique()
    if len(label_set) == 1:  # Only keep windows with a single label
        X.append(window[features].values.astype(np.float32))
        y.append(label_set[0])

X = np.array(X)
y = np.array(y)

# Dataset
class EMGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X)  # shape: (N, window, channels)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].permute(1, 0), self.y[idx]  # shape: (channels, window)

dataset = EMGDataset(X, y)

In [3]:
len(dataset)

2916

In [4]:
# Split data
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# CNN Model
# class EMGCNN(nn.Module):
#     def __init__(self, num_classes=17, num_channels=8):
#         super(EMGCNN, self).__init__()
#         self.conv1 = nn.Conv1d(num_channels, 64, kernel_size=3)
#         self.pool = nn.MaxPool1d(2)
#         self.fc1 = nn.Linear(64 * ((window_size - 2) // 2), 100)
#         self.fc2 = nn.Linear(100, num_classes)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))  # (B, 64, L)
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# Deep CNN Model
class EMGCNN(nn.Module):
    def __init__(self, num_classes=17, num_channels=8):
        super(EMGCNN, self).__init__()
        self.conv1 = nn.Conv1d(num_channels, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(256)

        self.pool = nn.MaxPool1d(kernel_size=2)
        
        # Calculate output size after convolutions and pooling
        conv_output_size = window_size
        for _ in range(3):  # 3 conv + pool layers
            conv_output_size = conv_output_size // 2  # MaxPool1d halves it

        self.fc1 = nn.Linear(256 * conv_output_size, 128)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # -> (B, 64, L/2)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # -> (B, 128, L/4)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # -> (B, 256, L/8)

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
# Train setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EMGCNN(num_classes=len(mapg)).to(device)
criterion = nn.CrossEntropyLoss()


In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Training
for epoch in range(100):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    # print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

    # Validation
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    print(f"Train Accuracy: {100 * correct / total:.2f}%")

    # Validation
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Train Accuracy: 92.15%
Validation Accuracy: 66.78%
Train Accuracy: 79.63%
Validation Accuracy: 57.53%
Train Accuracy: 90.78%
Validation Accuracy: 65.41%
Train Accuracy: 90.31%
Validation Accuracy: 65.24%
Train Accuracy: 92.62%
Validation Accuracy: 67.81%
Train Accuracy: 94.68%
Validation Accuracy: 69.69%
Train Accuracy: 82.72%
Validation Accuracy: 56.68%
Train Accuracy: 82.12%
Validation Accuracy: 59.42%
Train Accuracy: 93.01%
Validation Accuracy: 70.55%
Train Accuracy: 90.31%
Validation Accuracy: 67.12%
Train Accuracy: 82.80%
Validation Accuracy: 61.82%
Train Accuracy: 68.01%
Validation Accuracy: 46.40%
Train Accuracy: 89.19%
Validation Accuracy: 62.67%
Train Accuracy: 87.39%
Validation Accuracy: 62.67%
Train Accuracy: 87.91%
Validation Accuracy: 61.82%
Train Accuracy: 90.14%
Validation Accuracy: 64.04%
Train Accuracy: 89.62%
Validation Accuracy: 62.84%
Train Accuracy: 85.25%
Validation Accuracy: 63.36%
Train Accuracy: 90.69%
Validation Accuracy: 65.24%
Train Accuracy: 90.74%
Validati